In [7]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from langgraph.graph.message import MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
from typing import Callable
import os

In [8]:
class AgentsState(MessagesState):
    current_agent: str
    transfered_by: str


load_dotenv(dotenv_path="../../.env")
llm = init_chat_model(
    "ollama:qwen3-vl:32b",
    base_url=os.environ.get("OLLAMA_BASE_URL"),
)

In [9]:
def make_agent(prompt: str, tools: list[Callable]):
    def agent_node(state: AgentsState):
        llm_with_tools = llm.bind_tools(tools)
        response = llm_with_tools.invoke(
            f"""
            {prompt}

            Conversation History:
            {state["messages"]}
            
            """
        )

        return {
            "messages": [response],
        }

    agent_builder = StateGraph(AgentsState)

    agent_builder.add_node("agent", agent_node)
    agent_builder.add_node(
        "tools",
        ToolNode(tools=tools),
    )

    agent_builder.add_edge(START, "agent")
    agent_builder.add_conditional_edges("agent", tools_condition)
    agent_builder.add_edge("tools", "agent")
    agent_builder.add_edge("agent", END)
    return agent_builder.compile()

In [10]:
@tool
def handoff_tool(transfer_to: str, transfered_by: str):
    """
    Handoff to another agent.

    Use this tool when the customer speaks a language that you don't understand.

    Possible values for `transfer_to`:
    - `korean_agent`
    - `greek_agent`
    - `spanish_agent`

    Possible values for `transfered_by`:
    - `korean_agent`
    - `greek_agent`
    - `spanish_agent`

    Args:
        transfer_to: The agent to transfer the conversation to
        transfered_by: The agent that transferred the conversation
    """
    return Command(
        update={
            "current_agent": transfer_to,
            "transfered_by": transfered_by,
        },
        goto=transfer_to,
        graph=Command.PARENT,
    )

In [ ]:
graph_builder = StateGraph(AgentsState)
graph_builder.add_node(
    "korean_agent",
    make_agent(
        prompt="You're a Korean customer support agent. You only speak and understand Korean.",
        tools=[handoff_tool],
    ),
    destinations=("greek_agent", "spanish_agent"),
)
graph_builder.add_node(
    "greek_agent",
    make_agent(
        prompt="You're a Greek customer support agent. You only speak and understand Greek.",
        tools=[handoff_tool],
    ),
    destinations=("korea_agent", "spanish_agent"),
)
graph_builder.add_node(
    "spanish_agent",
    make_agent(
        prompt="You're a Spanish customer support agent. You only speak and understand Spanish.",
        tools=[handoff_tool],
    ),
    destinations=("greek_agent", "korean_agent"),
)

graph_builder.add_edge(START, "korean_agent")

In [13]:
graph = graph_builder.compile()
for event in graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "Hola~! Necesito ayuda con mi cuenta.",
            }
        ]
    },
    stream_mode="values",
):
    print(event)

{'messages': [HumanMessage(content='Hola~! Necesito ayuda con mi cuenta.', additional_kwargs={}, response_metadata={}, id='05e0ed80-9994-4332-a7ca-e101c9f31420')]}
{'messages': [HumanMessage(content='Hola~! Necesito ayuda con mi cuenta.', additional_kwargs={}, response_metadata={}, id='05e0ed80-9994-4332-a7ca-e101c9f31420')], 'current_agent': 'spanish_agent', 'transfered_by': 'korean_agent'}
{'messages': [HumanMessage(content='Hola~! Necesito ayuda con mi cuenta.', additional_kwargs={}, response_metadata={}, id='05e0ed80-9994-4332-a7ca-e101c9f31420'), AIMessage(content='The user message is in Spanish, and as a Spanish customer support agent, I can handle this directly. No handoff is necessary. \n\nNo tool call required.', additional_kwargs={}, response_metadata={'model': 'qwen3-vl:32b', 'created_at': '2025-12-03T07:42:49.923475023Z', 'done': True, 'done_reason': 'stop', 'total_duration': 15330294940, 'load_duration': 70706897, 'prompt_eval_count': 355, 'prompt_eval_duration': 67704584,